# K-Nearest Neighbors Algorithim:  Python Class Implementation


In [19]:
# import libraries
import requests
from contextlib import closing
import csv
import pandas as pd
from random import seed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler




In [4]:
cols = ['sepal_length', ' sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv('IRIS.csv', names=cols, skiprows=1)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
df.describe()


,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1    sepal_width  150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [22]:
df['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [23]:
class KNN:

    def __init__(self, name):
        self.name = name
        

    def load_csv(file_name):
        """
        load CSV file into pandas dataframe
        """
        dataframe = pd.read_csv(file_name, names=cols, skiprows=1)

        return dataframe
    

    # Data Wrangling
    def string_to_float(dataframe, col):
        """
        convert string column values to floats;
        not needed -> values are floats   
        """
        pass


    def ints_to_floats(dataframe, col):
        """
        convert integers values to floats;
        not needed -> values are floats  
        """
        pass
    
    
    def create_unique_class_id(dataframe, col):
        """
        identify unique classification values; 
        convert column values to integers
        """
        s = pd.Series(dataframe[col])
        cat = pd.Categorical(['1', '2', '3'], categories=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
        class_val_labels, class_uniques = pd.factorize(cat)

        return class_val_labels, class_uniques

   
    
    # Pre-Processing
    def train_test_split(dataframe, col1, col2):
        """
        Split arrays or matrices into random train and test subsets
        """
        X = dataframe[col1]
        y = dataframe[col2]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
        return X_train, X_test, y_train, y_test
    
    
    def normalize():
        """
        rescale dataset columns to range 0-1;
        must be called after train/test split to avoid leakage/bias
        """
        data = [X_train]
        scaler = MinMaxScaler()

        return scaler.fit.transform(data)
    

    def calculate_ed():
        """
        calculate euclidean distance between two vectors
        """
        pass
    

    # def min_max(dataframe):
    #     """
    #     find the minimum and maximum values for each column;
    #     create a list of these values
    #     """
    #     # create empty list
    #     minmax = []

    #     # groups the data by columns and assigns the value of 
    #     # the minimum value per group to the min variable
    #     min = dataframe.groupby(col).min()

    #     # groups the data by columns and assigns the value of
    #     # the maximum value per group to the max variable
    #     max = dataframe.groupby(col).max()

    #     for i in range(len(dataframe[0])):
    #         minmax.append([min, max])

    #     return minmax

    ## Note: can use MinMaxScaler fit to accomplish this and then transform to scale for normalization

    # def mean():
    #     """
    #     groups the data by columns and returns the mean value per group
    #     """
    #     return dataframe.groupby(col).mean()

    
    # Implement K-NN Algorithm
    def get_neighbors():
        """
        locate most similar neighbors
        """
        pass
    

    def predict_class():
        """
        make a prediction with neighbors
        """
        pass
    

    def k_nn():
        """
        K-NN algorithm
        """
        pass

    
    # Evaluate K-NN Algorithm
    def cross_val_split():
        """
        split dataset into k folds
        """
        pass
    

    def accuracy_metric():
        """
        calculate accuracy percentage
        """
        pass
    

    def cross_val_metric():
        """
        evaluate algorithm using cross validation 
        """
        pass



In [27]:
# Test KNN class
knn = KNN("K-NN")
dataframe = load_csv('IRIS.csv')
print(dataframe.head())
print(dataframe.tail())
dataframe.info()
create_unique_class_id(dataframe, "class")
print(class_val_labels)

   sepal_length   sepal_width  petal_length  petal_width        class
0           5.1           3.5           1.4          0.2  Iris-setosa
1           4.9           3.0           1.4          0.2  Iris-setosa
2           4.7           3.2           1.3          0.2  Iris-setosa
3           4.6           3.1           1.5          0.2  Iris-setosa
4           5.0           3.6           1.4          0.2  Iris-setosa
     sepal_length   sepal_width  petal_length  petal_width           class
145           6.7           3.0           5.2          2.3  Iris-virginica
146           6.3           2.5           5.0          1.9  Iris-virginica
147           6.5           3.0           5.2          2.0  Iris-virginica
148           6.2           3.4           5.4          2.3  Iris-virginica
149           5.9           3.0           5.1          1.8  Iris-virginica
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Nul

NameError: name 'create_unique_class_id' is not defined